# コンペ概要の理解（ドラフト）

> 公式ページ: https://www.kaggle.com/competitions/hull-tactical-market-prediction

- タスク: 市場に関する将来リターンの予測（詳細な定義・評価指標は公式ページを参照）
- データ構造（手元のtrain/testから推定）:
  - 時系列キー: `date_id`
  - 特徴量群: `D*`, `E*`, `I*`, `M*`, `P*`, `S*`, `V*` などの多数の数値列
  - trainのみ: 目的変数候補 `market_forward_excess_returns` または `forward_returns`、関連指標に `risk_free_rate`
  - test: `is_scored`（採点対象行フラグ）と `lagged_*` 系の説明変数

このノートでは、前処理・特徴量概要・欠損や分布の把握などのEDAを進め、
リーク回避と適切な時系列分割の方針を固めていきます。

## 特徴量（公式の説明要約）

> 参考: https://www.kaggle.com/competitions/hull-tactical-market-prediction/data

train.csv（過去の市場データ。カバレッジは数十年に及ぶため、初期期間は欠損が多い点に注意）:

- date_id: 単一の取引日を表す識別子
- M*: Market Dynamics/Technical features（市場ダイナミクス/テクニカル）
- E*: Macro Economic features（マクロ経済）
- I*: Interest Rate features（金利）
- P*: Price/Valuation features（価格/バリュエーション）
- V*: Volatility features（ボラティリティ）
- S*: Sentiment features（センチメント）
- MOM*: Momentum features（モメンタム）
- D*: Dummy/Binary features（二値ダミー）
- forward_returns: S&P 500 を買って翌日売った場合のリターン（train のみ）
- risk_free_rate: フェデラル・ファンド・レート（train のみ）
- market_forward_excess_returns: 期待に対する将来リターンの超過分（5年ローリング平均を差し引き、MAD=4基準でウィンザー化；train のみ）

test.csv（本番と同構造のモック。公開LBは train の最後の180日をコピーしたもので意味が限定的。評価APIが配布する非公開テストは学習期間中に更新され得る）:

- date_id
- [feature_name]: train.csv と同一の特徴列
- is_scored: 評価指標の計算に含めるか（学習フェーズでは先頭180行のみ true；test のみ）
- lagged_forward_returns: forward_returns の1日ラグ
- lagged_risk_free_rate: risk_free_rate の1日ラグ
- lagged_market_forward_excess_returns: market_forward_excess_returns の1日ラグ（同様の処理で算出）

# EDA 概要とデータ方針

このノートでは Hull Tactical Market Prediction のEDAを行います。

- データはGitに含めません。Kaggle APIで毎回取得します。
- 取得手順は `README.md` の「データ運用ポリシー（EDA）」を参照してください。
- 取得スクリプト: `scripts/fetch_data.sh`（事前に `chmod +x` で実行権限を付与）

データ配置（ローカル専用）:
- `data/raw/`: 公式zipの展開先
- `data/interim/`, `data/processed/`, `data/external/`: 必要に応じて中間生成物を配置（Git管理しない）
- `artifacts/`: 学習成果物（Git管理しない）


In [ ]:
# データの読み込み例（ファイル名は実データに合わせて調整してください）
import pandas as pd

train_path = "data/raw/train.parquet"
test_path = "data/raw/test.parquet"

try:
    train = pd.read_parquet(train_path)
    test = pd.read_parquet(test_path)
    display(train.info())
    display(train.head())
except FileNotFoundError as e:
    print("[info] ローカルにデータが見つかりません。先に以下を実行してください:")
    print("  ./scripts/fetch_data.sh")
    raise e

In [ ]:
# 特徴量グループの件数（ヘッダから集計）
import csv
from collections import Counter
from pathlib import Path

def count_feature_groups(header):
    special = {'date_id','forward_returns','risk_free_rate','market_forward_excess_returns','is_scored'}
    counts = Counter()
    for c in header:
        if not c or c in special or c.startswith('lagged_'):
            continue
        prefix = c[0] if c else None
        if prefix and prefix.isalpha():
            counts[prefix] += 1
    return counts

# train
with Path('data/raw/train.csv').open('r', newline='') as f:
    header_train = next(csv.reader(f))
counts_train = count_feature_groups(header_train)
print('train groups:', dict(sorted(counts_train.items())))

# test
with Path('data/raw/test.csv').open('r', newline='') as f:
    header_test = next(csv.reader(f))
counts_test = count_feature_groups(header_test)
print('test groups:', dict(sorted(counts_test.items())))
print('test lagged count:', sum(1 for c in header_test if c.startswith('lagged_')))

## 目的変数・MAD・ウィンザー化（要点）

- 目的変数: `market_forward_excess_returns`（翌日超過リターンから5年ローリング平均を差し引き、MAD=4でウィンザー化済み）
- 超過リターン: r_excess = forward_returns − risk_free_rate
- デトレンド: 5年ローリング平均（過去のみ）を引いて上振れ/下振れに変換
- ウィンザー化（MAD=4）: 中央値 m と MAD を用い、[m−4×MAD, m+4×MAD] にクリップ（外れ値を境界に置換）
- 評価: test の is_scored==true 行に対する RMSE
- リーク防止: ローリング統計は shift(1) で「その時点直前まで」のデータに限定して算出する

In [ ]:
# MAD=4 のウィンザー化デモ（リーク防止の rolling+shift 例）
import pandas as pd
import numpy as np
from pathlib import Path

# デモ方針: 実データが無ければ合成データで挙動を確認（実データがあればそれを使用）
train_csv = Path('data/raw/train.csv')

def compute_mad_winsorized(series: pd.Series, k: float = 4.0) -> pd.Series:
    """
    series: 時系列データ（時刻順に並んでいる前提）
    k: MAD 係数（k=4なら MAD=4）
    処理: 中央値m_t と MAD_t を、各時点tで『t-1まで』のデータから求め、[m_t-k*MAD_t, m_t+k*MAD_t] にクリップ
    """
    s = series.astype(float).copy()
    # 過去のみで中央値とMADを計算
    med = s.shift(1).expanding().median()
    abs_dev = (s.shift(1) - med).abs()
    mad = abs_dev.expanding().median()
    lower = med - k * mad
    upper = med + k * mad
    return s.clip(lower=lower, upper=upper)

if train_csv.exists():
    df = pd.read_csv(train_csv, usecols=['date_id','forward_returns','risk_free_rate','market_forward_excess_returns'])
    df = df.sort_values('date_id').reset_index(drop=True)
    r_excess = df['forward_returns'] - df['risk_free_rate']
    # ここでは 5年ローリング平均の再現は行わず（提供済みラベルを使用する想定）、
    # MAD=4のウィンザー化処理のデモのみを示す。
    y = df['market_forward_excess_returns']
    y_wins = compute_mad_winsorized(y, k=4.0)
    print('データあり: y と winsorized(y) の基本統計')
    print(pd.DataFrame({'y': y.describe(), 'y_wins': y_wins.describe()}))
else:
    # 合成データ: 正規 + 外れ値
    rng = np.random.default_rng(0)
    base = rng.normal(loc=0.0, scale=1.0, size=500)
    base[::50] += 8  # 外れ値を混ぜる
    s = pd.Series(base)
    s_wins = compute_mad_winsorized(s, k=4.0)
    print('合成データ: before/after の標準偏差比較')
    print({'std_before': float(s.std()), 'std_after': float(s_wins.std())})

## 公式評価コンポーネントの入出力（要点）

- データ供給: `test.csv` を行ID列（既定は `batch_id`、無ければ先頭列）でバッチ分割して `predict` に渡す
- 予測の検証: 返す予測の行数は、受け取った row_ids と一致していなければならない（不一致はエラー）
- 提出生成: 予測と row_ids を結合し、`submission.parquet` を作成（行ID列を先頭に配置、予測列名は既定 `prediction`）
- タイムアウト: 応答は1バッチあたり最大5分。サーバ起動は最大15分で検知
- 評価: Kaggle 側で `is_scored==true` の行に対して RMSE を計算（提出自体は全行が対象）